# Model Training Example

## Imports and setup

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from typing import List
from custom_models import PromoterType

In [ ]:
from Bio import SeqIO
from pathlib import Path

cwd = Path.cwd()
benchmark_path = cwd / "Dataset" / "Benchmark_dataset"
independent_path = cwd / "Dataset" / "Independent_test_dataset"

#Read Sequence file in fasta format using BioPython library 
    
super_transform = {"70": PromoterType.SIGMA_70, 
                   "54": PromoterType.SIGMA_54, 
                   "38": PromoterType.SIGMA_38, 
                   "32": PromoterType.SIGMA_32, 
                   "28": PromoterType.SIGMA_28, 
                   "24": PromoterType.SIGMA_24                   
                  }
sequences = []
labels = []
for record in SeqIO.parse(independent_path / "independent.txt", "fasta"):
    seq = str(record.seq)
    label = super_transform[record.id]
    sequences.append(seq)
    labels.append(label)

## Data loading

In [ ]:
with open(benchmark_path / 'promoter_and_non-promoter' / 'positive2860.txt') as handle:
    promoters = [str(record.seq) for record in SeqIO.parse(handle, "fasta")]
with open(benchmark_path / 'promoter_and_non-promoter' / 'negative2860.txt') as handle:
    non_promoters = [str(record.seq) for record in SeqIO.parse(handle, "fasta")]
with open(benchmark_path / 'sigma_subtypes' / 'sigma24promoter.txt') as handle:
    sigma24promoter = [str(record.seq) for record in SeqIO.parse(handle, "fasta")]
with open(benchmark_path / 'sigma_subtypes' / 'sigma28promoter.txt') as handle:
    sigma28promoter = [str(record.seq) for record in SeqIO.parse(handle, "fasta")]
with open(benchmark_path / 'sigma_subtypes' / 'sigma32promoter.txt') as handle:
    sigma32promoter = [str(record.seq) for record in SeqIO.parse(handle, "fasta")]
with open(benchmark_path / 'sigma_subtypes' / 'sigma38promoter.txt') as handle:
    sigma38promoter = [str(record.seq) for record in SeqIO.parse(handle, "fasta")]
with open(benchmark_path / 'sigma_subtypes' / 'sigma54promoter.txt') as handle:
    sigma54promoter = [str(record.seq) for record in SeqIO.parse(handle, "fasta")]
with open(benchmark_path / 'sigma_subtypes' / 'sigma70promoter.txt') as handle:
    sigma70promoter = [str(record.seq) for record in SeqIO.parse(handle, "fasta")]

# Promoter multiclass
benchmark_dataset = sigma24promoter + sigma28promoter + sigma32promoter + sigma38promoter + sigma54promoter + sigma70promoter
benchmark_labels = [PromoterType.SIGMA_24]*len(sigma24promoter) + [PromoterType.SIGMA_28]*len(sigma28promoter) + [PromoterType.SIGMA_32]*len(sigma32promoter) + [PromoterType.SIGMA_38]*len(sigma38promoter) + [PromoterType.SIGMA_54]*len(sigma54promoter) + [PromoterType.SIGMA_70]*len(sigma70promoter)
benchmark_labels_ints = [_type.value - 2 for _type in benchmark_labels]
benchmark_labels_one_hot = tf.one_hot(benchmark_labels_ints, 6)
# Promoter binary (first stage)
binary_dataset = promoters + non_promoters
binary_labels_int = [1]*len(promoters) + [0]*len(non_promoters)
binary_labels_one_hot = tf.one_hot(binary_labels_int, 2)

In [ ]:
def preprocess_seqs(data: List[str]) -> List[np.ndarray]:
    ## Check Seqs Length
    for i, seq in enumerate(data):   
        if ( seq_len := len(seq) ) != 81:
            raise ValueError(f"Each sequence must have a length of 81nt.\nSequence {i} has length {seq_len}nt")

    encoding = {'A':[1,0,0,0],'T':[0,1,0,0],'C':[0,0,1,0],'G':[0,0,0,1]}
    return [np.array([[encoding[x] for x in seq.upper()] for seq in data])]

In [ ]:
training_data = {}
current_order = "hybrid" # Just a name, it's used as a prefix when making the checkpoints' folders

### For One-Hot models

In [ ]:
training_data[current_order] = {
    "IsPromoter":    {"input": preprocess_seqs(binary_dataset),    "output": binary_labels_one_hot.numpy()},
    "PromoterType": {"input": preprocess_seqs(benchmark_dataset), "output": benchmark_labels_one_hot.numpy()}
}

## Model definition

In [ ]:
# import necessary layers  
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization
from tensorflow.keras.layers import MaxPooling1D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model, regularizers

from tensorflow_addons.optimizers import LazyAdam

tf.keras.backend.clear_session()

models = [{"name": "IsPromoter", "in_prefix": "", "out_classes": 2},
          {"name": "PromoterType", "in_prefix": "", "out_classes": 6}]

def generate_model(name: str = "Latest", in_prefix: str = '', out_classes: int = 6):
    # input
    input_ = Input(shape =(81,4), name=in_prefix + 'mono_mer_in')

    # 1st Conv Block
    # Params for first Conv1D
    hp_filters_1 = 128
    hp_kernel_1 = 5
    hp_kreg_1 = 1e-3
    hp_breg_1 = 1e-2
    # Params for second Conv1D
    hp_filters_2 = 128
    hp_kernel_2 = 9
    hp_kreg_2 = 1e-3
    hp_breg_2 = 1e-5
    # Params for Dropout
    hp_drop_1 = 0.45
    
    x = Conv1D (filters=hp_filters_1, kernel_size=hp_kernel_1, padding ='same', activation='relu', kernel_regularizer = regularizers.l2(hp_kreg_1), bias_regularizer = regularizers.l2(hp_breg_1))(input_)
    x = Conv1D (filters=hp_filters_2, kernel_size=hp_kernel_2, padding ='same', activation='relu', kernel_regularizer = regularizers.l2(hp_kreg_2), bias_regularizer = regularizers.l2(hp_breg_2))(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size =2, strides =2, padding ='same')(x)
    x = Dropout(rate=hp_drop_1)(x)

    # Fully connected layers
    x = Flatten()(x)
    hp_units = 32
    x = Dense(units=hp_units, activation ='relu', kernel_regularizer = regularizers.l2(1e-3), bias_regularizer = regularizers.l2(1e-3))(x)
    output = Dense(units = out_classes, activation ='softmax')(x)

    # Creating the model
    model = Model (inputs=input_, outputs =output, name=name)
    model.compile(optimizer=LazyAdam(), loss=tfa.losses.SigmoidFocalCrossEntropy(alpha=0.20, gamma=3, reduction=tf.keras.losses.Reduction.AUTO), metrics=['accuracy'])
    
    return model

# Model Training - KFold

In [ ]:
from tensorflow.keras.models import load_model
from sklearn.model_selection import StratifiedKFold
import pathlib

In [ ]:
%%time
FOLDS = 5
EPOCHS = 100
VERBOSITY = 1
BATCH_SIZE = None

version_id = "_1"

best_by_loss = []

kfold = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=0)

early_stop_callback = tf.keras.callbacks.EarlyStopping(
                                    monitor="val_loss",
                                    min_delta=0,
                                    patience=15,
                                    verbose=0,
                                    mode="auto",
                                    restore_best_weights=True,
                                )

for current_model in models:
    current_name = current_model["name"]
    inputs = training_data[current_order][current_name]["input"]
    targets = training_data[current_order][current_name]["output"]

    log_dir_base = pathlib.Path.cwd() / f"logs_{current_order}{version_id}"
    checkpoint_dir_base = pathlib.Path.cwd() / f"checkpoints_{current_order}{version_id}"

    # Save fold history for future plotting
    fold_history = []

    # Define per-fold score containers
    acc_per_fold = []
    loss_per_fold = []

    # K-fold Cross Validation model evaluation
    fold_no = 1
    for train, test in kfold.split(np.zeros(len(targets)), targets.argmax(1)):

        # Different path for each fold
        fold_name = current_name + '_fold_' + str(fold_no)
        log_dir = log_dir_base / fold_name
        checkpoint_dir = checkpoint_dir_base / fold_name

        # Make sure those paths exist
        log_dir.mkdir(parents=True, exist_ok=True)
        checkpoint_dir.mkdir(parents=True, exist_ok=True)

        # Callbacks for logging and model saving
        logger_callback = tf.keras.callbacks.CSVLogger(log_dir / "history.csv", append=False)
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_dir,
            save_weights_only=False, 
            monitor='val_loss', 
            mode='min', 
            save_best_only=True)

        # Define the model architecture
        model = generate_model(**current_model)

        # Print status
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')

        # Fit data to model
        history = model.fit([inputs[i][train] for i in range(len(inputs))][0], targets[train], # [0] added
                  batch_size=BATCH_SIZE, 
                  epochs=EPOCHS, 
                  verbose=VERBOSITY, 
                  callbacks=[logger_callback, model_checkpoint_callback, early_stop_callback], 
                  validation_data=([inputs[i][test] for i in range(len(inputs))][0], targets[test])) # [0] added

        fold_history.append(history)
        
        #print("Finished training fold!!!")

        # Load best checkpoint of the fold
        model = load_model(checkpoint_dir)

        # Generate generalization metrics
        scores = model.evaluate([inputs[i][test] for i in range(len(inputs))][0], targets[test], verbose=0) # [0] added
        print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
        acc_per_fold.append(scores[1] * 100)
        loss_per_fold.append(scores[0])

        # Increase fold number
        fold_no = fold_no + 1

        # Free resources
        tf.keras.backend.clear_session()
        
    # == Provide average scores ==
    print('\n\n\n\n')
    print('------------------------------------------------------------------------')
    print(f'Score per fold -- {current_name}')
    for i in range(0, len(acc_per_fold)):
        print('------------------------------------------------------------------------')
        print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
    print('------------------------------------------------------------------------')
    print('Average scores for all folds:')
    print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
    print(f'> Loss: {np.mean(loss_per_fold)}')
    print('------------------------------------------------------------------------')
    print('\n\n\n\n')

    # For future loading
    best_by_loss.append(np.argmin(loss_per_fold))
print(best_by_loss)